In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=81119f8cc172cab229ff4116730577de4beae287de623b50df92e6be208b70c0
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf
from google.colab import drive

In [5]:
spark=SparkSession.builder.appName('data_processing_c').getOrCreate()

In [6]:
drive.mount('/content/drive')
df=spark.read.csv('/content/drive/My Drive/covid19/Casos_positivos_de_COVID-19_en_Colombia-100K.csv',inferSchema=True,header=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df.columns

['fecha reporte web',
 'ID de caso',
 'Fecha de notificación',
 'Código DIVIPOLA departamento',
 'Nombre departamento',
 'Código DIVIPOLA municipio',
 'Nombre municipio',
 'Edad',
 'Unidad de medida de edad',
 'Sexo',
 'Tipo de contagio',
 'Ubicación del caso',
 'Estado',
 'Código ISO del país',
 'Nombre del país',
 'Recuperado',
 'Fecha de inicio de síntomas',
 'Fecha de muerte',
 'Fecha de diagnóstico',
 'Fecha de recuperación',
 'Tipo de recuperación',
 'Pertenencia étnica',
 'Nombre del grupo étnico']

In [8]:
df.printSchema()

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [9]:
df.select('Fecha de muerte','Nombre departamento','Sexo').show(10)

+---------------+-------------------+----+
|Fecha de muerte|Nombre departamento|Sexo|
+---------------+-------------------+----+
|           null|             BOGOTA|   F|
|           null|              VALLE|   M|
|           null|          ANTIOQUIA|   F|
|           null|          ANTIOQUIA|   M|
|           null|          ANTIOQUIA|   M|
|           null|          ANTIOQUIA|   F|
|           null|          CARTAGENA|   F|
|           null|             BOGOTA|   F|
|           null|             BOGOTA|   F|
|           null|             BOGOTA|   F|
+---------------+-------------------+----+
only showing top 10 rows



In [10]:
df =df.withColumnRenamed("fecha reporte web", "fecha_reporte_web")
df =df.withColumnRenamed("ID de caso", "id_caso")
df =df.withColumnRenamed("Fecha de notificación", "fecha_notificacion")
df =df.withColumnRenamed("Código DIVIPOLA departamento", "codigo_departamento")
df =df.withColumnRenamed("Nombre departamento", "departamento")
df =df.withColumnRenamed("Código DIVIPOLA municipio", "codigo_municipio")
df =df.withColumnRenamed("Nombre municipio", "municipio")
df =df.withColumnRenamed("Edad", "edad")
df =df.withColumnRenamed("Sexo", "sexo")
df =df.withColumnRenamed("Tipo de contagio", "tipo_contagio")
df =df.withColumnRenamed("Ubicación del caso", "ubicacion_caso")
df =df.withColumnRenamed("Estado", "estado")
df =df.withColumnRenamed("Código ISO del país", "codigo_pais")
df =df.withColumnRenamed("Nombre del país", "nombre_pais")
df =df.withColumnRenamed("Recuperado", "recuperado")
df =df.withColumnRenamed("Fecha de inicio de síntomas", "fecha_inicio_sintomas")
df =df.withColumnRenamed("Fecha de muerte", "fecha_muerte")
df =df.withColumnRenamed("Fecha de diagnóstico", "fecha_diagnostico")
df =df.withColumnRenamed("Fecha de recuperación", "fecha_recuperacion")
df =df.withColumnRenamed("Tipo de recuperación", "tipo_recuperacion")
df =df.withColumnRenamed("Pertenencia étnica", "pertenencia_etnica")
df =df.withColumnRenamed("Nombre del grupo étnico", "nombre_etnia")

In [11]:
df = df.withColumn("estado_por_sexo", concat(col("id_caso"), lit(", "), col("estado"), lit(", "), col("sexo")))
df.select("estado_por_sexo").show(5)

+---------------+
|estado_por_sexo|
+---------------+
|     1, Leve, F|
|     2, Leve, M|
|     3, Leve, F|
|     4, Leve, M|
|     5, Leve, M|
+---------------+
only showing top 5 rows



In [12]:
df = df.drop("fecha_muerte","codigo_municipio")
df.columns

['fecha_reporte_web',
 'id_caso',
 'fecha_notificacion',
 'codigo_departamento',
 'departamento',
 'municipio',
 'edad',
 'Unidad de medida de edad',
 'sexo',
 'tipo_contagio',
 'ubicacion_caso',
 'estado',
 'codigo_pais',
 'nombre_pais',
 'recuperado',
 'fecha_inicio_sintomas',
 'fecha_diagnostico',
 'fecha_recuperacion',
 'tipo_recuperacion',
 'pertenencia_etnica',
 'nombre_etnia',
 'estado_por_sexo']

In [13]:
df.select("id_caso", "estado","sexo").filter(df["sexo"] == "F").show(10)

+-------+------+----+
|id_caso|estado|sexo|
+-------+------+----+
|      1|  Leve|   F|
|      3|  Leve|   F|
|      6|  Leve|   F|
|      7|  Leve|   F|
|      8|  Leve|   F|
|      9|  Leve|   F|
|     10|  Leve|   F|
|     11|  Leve|   F|
|     12|  Leve|   F|
|     13|  Leve|   F|
+-------+------+----+
only showing top 10 rows



In [14]:
lambda_function = udf(lambda column_value: "estado: " + column_value)
df = df.withColumn("tag", lambda_function(df["estado"]))
df.select('estado','tag').show(5)

+------+------------+
|estado|         tag|
+------+------------+
|  Leve|estado: Leve|
|  Leve|estado: Leve|
|  Leve|estado: Leve|
|  Leve|estado: Leve|
|  Leve|estado: Leve|
+------+------------+
only showing top 5 rows



In [15]:
departamentos = df.groupBy("departamento").count().orderBy(desc("count")).limit(10)
departamentos.show()

+------------+-----+
|departamento|count|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   ATLANTICO|10994|
|       VALLE|10404|
|   CARTAGENA| 8333|
|   ANTIOQUIA| 4554|
|      NARIÑO| 3520|
|CUNDINAMARCA| 2827|
|    AMAZONAS| 2317|
|       CHOCO| 1636|
+------------+-----+



In [16]:
df.createOrReplaceTempView("data")
departamentos = spark.sql("SELECT departamento, COUNT(*) AS casos FROM data GROUP BY departamento ORDER BY casos DESC LIMIT 10")
departamentos.show()

+------------+-----+
|departamento|casos|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   ATLANTICO|10994|
|       VALLE|10404|
|   CARTAGENA| 8333|
|   ANTIOQUIA| 4554|
|      NARIÑO| 3520|
|CUNDINAMARCA| 2827|
|    AMAZONAS| 2317|
|       CHOCO| 1636|
+------------+-----+



In [17]:
df.createOrReplaceTempView("data")
departamentos = spark.sql("SELECT departamento, COUNT(*) AS casos FROM data GROUP BY departamento ORDER BY casos DESC LIMIT 10")
departamentos.show()

+------------+-----+
|departamento|casos|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   ATLANTICO|10994|
|       VALLE|10404|
|   CARTAGENA| 8333|
|   ANTIOQUIA| 4554|
|      NARIÑO| 3520|
|CUNDINAMARCA| 2827|
|    AMAZONAS| 2317|
|       CHOCO| 1636|
+------------+-----+



In [18]:
municipios = spark.sql("SELECT municipio, COUNT(*) AS casos FROM data GROUP BY municipio ORDER BY casos DESC LIMIT 10")
municipios.show()

+------------+-----+
|   municipio|casos|
+------------+-----+
|      BOGOTA|30016|
|BARRANQUILLA|13065|
|   CARTAGENA| 8333|
|        CALI| 7747|
|     SOLEDAD| 6233|
|     LETICIA| 2194|
|    MEDELLIN| 2137|
|      TUMACO| 1501|
|BUENAVENTURA| 1453|
|      QUIBDO| 1367|
+------------+-----+



In [19]:
dias = df.groupBy("fecha_reporte_web").count().orderBy(desc("count")).limit(10)
dias.show()

+-----------------+-----+
|fecha_reporte_web|count|
+-----------------+-----+
|27/6/2020 0:00:00| 4149|
|26/6/2020 0:00:00| 3843|
|24/6/2020 0:00:00| 3541|
|25/6/2020 0:00:00| 3486|
|29/6/2020 0:00:00| 3274|
|28/6/2020 0:00:00| 3178|
|18/6/2020 0:00:00| 3171|
|19/6/2020 0:00:00| 3059|
|21/6/2020 0:00:00| 3019|
|30/6/2020 0:00:00| 2803|
+-----------------+-----+



In [20]:
dias = spark.sql("SELECT fecha_reporte_web as Fecha, COUNT(*) AS Casos FROM data GROUP BY fecha_reporte_web ORDER BY casos DESC LIMIT 10")
dias.show()

+-----------------+-----+
|            Fecha|Casos|
+-----------------+-----+
|27/6/2020 0:00:00| 4149|
|26/6/2020 0:00:00| 3843|
|24/6/2020 0:00:00| 3541|
|25/6/2020 0:00:00| 3486|
|29/6/2020 0:00:00| 3274|
|28/6/2020 0:00:00| 3178|
|18/6/2020 0:00:00| 3171|
|19/6/2020 0:00:00| 3059|
|21/6/2020 0:00:00| 3019|
|30/6/2020 0:00:00| 2803|
+-----------------+-----+



In [21]:
edad = df.groupBy("edad").count().orderBy(desc("count")).limit(10)
edad.show()

+----+-----+
|edad|count|
+----+-----+
|  30| 2735|
|  29| 2611|
|  31| 2569|
|  28| 2540|
|  27| 2494|
|  26| 2436|
|  33| 2371|
|  32| 2362|
|  25| 2335|
|  34| 2310|
+----+-----+



In [22]:
edad = spark.sql("SELECT edad, COUNT(*) AS casos FROM data GROUP BY edad ORDER BY casos DESC LIMIT 10")
edad.show()

+----+-----+
|edad|casos|
+----+-----+
|  30| 2735|
|  29| 2611|
|  31| 2569|
|  28| 2540|
|  27| 2494|
|  26| 2436|
|  33| 2371|
|  32| 2362|
|  25| 2335|
|  34| 2310|
+----+-----+



In [23]:
estado = df.groupBy("estado").count().orderBy(desc("count")).limit(10)
estado.show()

+---------+-----+
|   estado|count|
+---------+-----+
|     Leve|94367|
|Fallecido| 4663|
|      N/A|  970|
+---------+-----+



In [24]:
estado = spark.sql("SELECT estado, COUNT(*) AS casos FROM data GROUP BY estado ORDER BY casos DESC LIMIT 10")
estado.show()

+---------+-----+
|   estado|casos|
+---------+-----+
|     Leve|94367|
|Fallecido| 4663|
|      N/A|  970|
+---------+-----+



In [25]:
departamentos.write.csv("/content/drive/My Drive/covid19/lab62/departamentos.csv")
municipios.write.csv("/content/drive/My Drive/covid19/lab62/municipios.csv")
dias.write.csv("/content/drive/My Drive/covid19/lab62/dias.csv")
edad.write.csv("/content/drive/My Drive/covid19/lab62/edad.csv")
estado.write.csv("/content/drive/My Drive/covid19/lab62/estado.csv")